In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

alpha = 0.95
t = 0
K = 10
delta = 1/252
T = 0.292
s0 = 152.51
k = 170
r = 0.119/100
kt = int((T-t)/delta)
sigma = 49.07/100
mu = 16.91/100
n = 400


x = np.random.normal((mu-sigma**2/2)*delta, sigma*delta**0.5, (kt, n))
s = pd.DataFrame(index=range(kt+1), columns=range(n))
s.iloc[0] = s0
for i in range(len(s.index)-1):
    s.iloc[i+1] = s.iloc[i]*np.exp(x[i])

d1 = pd.DataFrame(index=range(kt+1), columns=range(n))
d2 = pd.DataFrame(index=range(kt+1), columns=range(n))
c_bs = pd.DataFrame(index=range(kt+1), columns=range(n))
for i in range(kt+1):
    d1.iloc[i] = ((s.iloc[i]/k).map(lambda x: np.log(x))+((r+sigma**2/2)*(T-i*delta)))/(sigma*(T-i*delta)**0.5)
    d2.iloc[i] = d1.iloc[i]-sigma*(T-i*delta)**0.5
    c_bs.iloc[i] = s.iloc[i]*d1.iloc[i].map(lambda x: norm.cdf(x))-k*np.exp(-r*i*delta)*d2.iloc[i].map(lambda x: norm.cdf(x))

ht = pd.DataFrame(index=range(kt+1), columns=range(n))
for i in range(kt+1):
    ht.iloc[i] = d1.iloc[i].map(lambda x: norm.cdf(x)) 

vt = s.iloc[0:-1]*ht.iloc[0:-1]-c_bs.iloc[0:-1]
vt_next_ar = s.iloc[1:].values*ht.iloc[0:-1].values-c_bs.iloc[1:].values
vt_next = pd.DataFrame(index=range(kt), columns=range(n), data=vt_next_ar)
vt_reb = s.iloc[1:]*ht.iloc[1:]-c_bs.iloc[1:]
vt_reb.reset_index(drop=True, inplace=True)
yt = vt_reb - vt_next
vt_real_ar = s.iloc[1:].values*ht.iloc[0:-1].values-c_bs.iloc[1:].values+yt.values*np.exp(r*delta)
vt_real = pd.DataFrame(vt_real_ar)
df_loss = -vt_real.diff().dropna()
df_loss.reset_index(drop=True, inplace=True)

# One day VaR
VaR_every_day = []
for i in df_loss.columns:
    VaR_every_day.append(df_loss[i].quantile(0.95))

one_day_VaR = np.mean(VaR_every_day)

df_loss_10 = -vt_real.diff(periods=10).dropna()
VaR_every_10days = []
for i in df_loss_10.columns:
    VaR_every_10days.append(df_loss_10[i].quantile(0.95))
# Ten days VaR
ten_day_VaR = np.mean(VaR_every_10days)
# Ten days VaR
ten_day_VaR2 = one_day_VaR*(10**0.5)

#Print Results
dict_result = {"One day VaR": one_day_VaR, "Ten days VaR": ten_day_VaR, "Ten days VaR by Agg": ten_day_VaR2}
df_result = pd.DataFrame(dict_result, index=['Value'])
df_result

,One day VaR,Ten days VaR,Ten days VaR by Agg
Value,12.073367,33.381675,38.179338
